# Word Count Lab: Building a word count application
This lab intends to cover techinques in Spark for a simple word count application. In this lab, we will calculate the most common words in the Complete Works of William Shakespeare retrieved from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). This could be scaled in larger applications.

## Part 1: Creating a base DataFrame and performing operations

### (1a) Create a DataFrame
We can generate a base DataFrame from a Python list of tuples and the `sqlContext.createDataFrame` method.

In [2]:
wordsDF = sqlContext.createDataFrame([('cat',), ('elephant',), ('rat',), ('rat',), ('cat', )], ['word'])
wordsDF.show()
print(type(wordsDF))
wordsDF.printSchema()

+--------+
|    word|
+--------+
|     cat|
|elephant|
|     rat|
|     rat|
|     cat|
+--------+

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- word: string (nullable = true)



### (1b) Using DataFrame functions to add an 's'
We can use DataFrame functions to manipulate elements in a column. For example, we can add an 's' to every word in our base DataFrame. We first use `concat` function to combine multiple string columns to a single string column. Notice that `lit` function is used to convert a string literal to a column. 

In [4]:
from pyspark.sql.functions import lit, concat
pluralDF = wordsDF.select(concat(wordsDF.word, lit('s')).alias('word'))
pluralDF.show()

+---------+
|     word|
+---------+
|     cats|
|elephants|
|     rats|
|     rats|
|     cats|
+---------+



### (1c) Length of each word
We can also use the `length` function to compute the number of characters in each word.

In [5]:
from pyspark.sql.functions import length
pluralLengthsDF = pluralDF.select(length(pluralDF.word))
pluralLengthsDF.show()

+------------+
|length(word)|
+------------+
|           4|
|           9|
|           4|
|           4|
|           4|
+------------+



## Part 2: Counting with Spark SQL and DataFrames
There are multiple ways to compute the number of times a particular element appears in a specified column. The naive way is call `collect` on all the elements and counts them in the driver program. This method performs slowly when the dataset is terabytes. An efficient way is to apply data parallel operations.

### (2a) Using `groupby` and `count`
We can perform aggregation by `groupby` function and then use `count` on the GroupedData object to count the ocurrences in the groups. To count the number of each words, we apply `groupby` on the 'word' column and use count to get the numbers

In [7]:
wordCountsDF = (wordsDF
                .groupBy(wordsDF.word)
                .count()
)
wordCountsDF.show()

+--------+-----+
|    word|count|
+--------+-----+
|     rat|    2|
|     cat|    2|
|elephant|    1|
+--------+-----+



## Part 2: Finding unique words and a mean value
### (3a) Unique words

In [10]:
# Number of different words in our DataFrame
uniqueWordsCount = wordCountsDF.count()
print(uniqueWordsCount)

3


### (3b) Means of groups using DataFrames
We can find the mean number of occurrences of words in `wordCountDF`. Notice that nothing is passed in `groupBy`. This just makes a DataFrame to be a groupedData object so that aggregation functions can be applied.

In [11]:
# Average number of occurrence
averageCount = wordCountsDF.groupBy().mean('count').head()[0]
#wordCountsDF.groupBy().select(mean().alias("count"))
print(averageCount)

1.66666666667


## Part 4: Apply word count to a file 

### (4a) The `wordCount` function

In [13]:
# The wordCount function to compute number of each words in a DataFrame
def wordCount(wordListDF):
    assert(str(type(wordListDF)) == "<class 'pyspark.sql.dataframe.DataFrame'>")
    assert('word' in wordListDF.columns)
    return wordListDF.groupBy('word').count()
wordCount(wordsDF).show()

+--------+-----+
|    word|count|
+--------+-----+
|     rat|    2|
|     cat|    2|
|elephant|    1|
+--------+-----+



### (4a) Capitalization and punctuation
The real word documents generally contains captialized letters and punctuation. We can use regular expressions to convert uppercase letters to lowercase and remove all punctuation.

In [15]:
from pyspark.sql.functions import regexp_replace, trim, col, lower

# The removePunctuation function
def removePunctuation(column):
    return trim(regexp_replace(lower(column), '[^0-9a-z\s]', '')).alias('sentence')

# A testing DataFrame for removePunctuation
sentenceDF = sqlContext.createDataFrame([('Hi, you!',),
                                         (' No under_score',),
                                         (' *      Remove punctuation and spaces   *   ',),
                                        ],['sentence'])
sentenceDF.show(truncate=False)
(sentenceDF
 .select(removePunctuation(col('sentence')))
 .show(truncate=False)
)

+--------------------------------------------+
|sentence                                    |
+--------------------------------------------+
|Hi, you!                                    |
| No under_score                             |
| *      Remove punctuation and spaces   *   |
+--------------------------------------------+

+-----------------------------+
|sentence                     |
+-----------------------------+
|hi you                       |
|no underscore                |
|remove punctuation and spaces|
+-----------------------------+



### (4c) Load a text file
We will apply the `wordCount` function on the Complete Works of William Shakespeare from Project Gutenberg. First, we load the file from local to HDFS.
`hadoop fs -copyFromLocal /path/to/file/shakespeare.txt /target/path/on/cluster`
Then we use `sqlContext.read.text()` method to load a text file to a DataFrame. 


In [16]:
fileName = 'shakespeare.txt'
shakespeareDF = sqlContext.read.text(fileName).select(removePunctuation(col('value')))
shakespeareDF.show(15, truncate=False)

+-------------------------------------------------+
|sentence                                         |
+-------------------------------------------------+
|1609                                             |
|                                                 |
|the sonnets                                      |
|                                                 |
|by william shakespeare                           |
|                                                 |
|                                                 |
|                                                 |
|1                                                |
|from fairest creatures we desire increase        |
|that thereby beautys rose might never die        |
|but as the riper should by time decease          |
|his tender heir might bear his memory            |
|but thou contracted to thine own bright eyes     |
|feedst thy lights flame with selfsubstantial fuel|
+-------------------------------------------------+
only showing

### (4d) Words from lines
To apply the `wordCount` function, we have two extra steps,
* split each sentence into a list of words by its spaces
* filter out empty lines or words

We use `split` and `explode` methods to accomplish these two tasks.

In [18]:
from pyspark.sql.functions import split, explode
shakeWordsDF = (shakespeareDF
                .select(explode(split(shakespeareDF.sentence, ' ')).alias('word'))
                .where(col('word') != ''))
shakeWordsDF.show()
shakeWordsDFCount = shakeWordsDF.count()
print(shakeWordsDFCount)

+-----------+
|       word|
+-----------+
|       1609|
|        the|
|    sonnets|
|         by|
|    william|
|shakespeare|
|          1|
|       from|
|    fairest|
|  creatures|
|         we|
|     desire|
|   increase|
|       that|
|    thereby|
|    beautys|
|       rose|
|      might|
|      never|
|        die|
+-----------+
only showing top 20 rows

882996


### (4e) Count the words
Now we can use the `wordCount` function to perform word counts.

In [19]:
from pyspark.sql.functions import desc
topWordsAndCountsDF = wordCount(shakeWordsDF)
# Display the most commonly used words
topWordsAndCountsDF.orderBy(col('count').desc()).show(truncate=False)

+----+-----+
|word|count|
+----+-----+
|the |27361|
|and |26028|
|i   |20681|
|to  |19150|
|of  |17463|
|a   |14593|
|you |13615|
|my  |12481|
|in  |10956|
|that|10890|
|is  |9134 |
|not |8497 |
|with|7771 |
|me  |7769 |
|it  |7678 |
|for |7558 |
|his |6857 |
|be  |6857 |
|your|6655 |
|this|6602 |
+----+-----+
only showing top 20 rows



In [20]:
topWordsAndCountsDF.show()

+----------+-----+
|      word|count|
+----------+-----+
|       art|  915|
|      some| 1337|
|     those|  545|
|     still|  552|
|  painters|    1|
|      hope|  354|
|    travel|   33|
|     cures|    8|
|    ransom|   53|
|     spoil|   25|
|   tresses|    3|
|       few|   64|
| forgetful|    5|
|    harder|   11|
|  tripping|    6|
| soundness|    1|
|    waters|   27|
|occidental|    1|
|    marrow|    4|
|    distil|    4|
+----------+-----+
only showing top 20 rows

